In [ ]:
!pip install langchain_chroma langchain_groq langchain_core langchain_community langchain_text_splitters pypdf gradio sentence-transformers

In [ ]:
import os
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters.sentence_transformers import SentenceTransformersTokenTextSplitter
from langchain_chroma import Chroma

In [ ]:
# Create directory for persistent DB
os.makedirs('pharma_db', exist_ok=True)

In [ ]:
# Initialize embedding model and Chroma vector DB
embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
db = Chroma(collection_name='pharma_database', embedding_function=embedding_model, persist_directory='pharma_db')

In [ ]:
# Prompt template
PROMPT_TEMPLATE = """
You are a highly knowledgeable assistant specializing in pharmaceutical sciences.
Answer the question based only on the following context:
{context}

Answer the question based on the above context:
{question}

Use the provided context to answer the user's question accurately and concisely.
Don't justify your answers.
Don't give information not mentioned in the CONTEXT INFORMATION.
Do not say "according to the context" or "mentioned in the context" or similar.
"""
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
output_parser = StrOutputParser()

In [ ]:
# Helper function to format retrieved documents
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

In [ ]:
# Function to process uploaded PDF files
def process_documents(file_paths):
    for file_path in file_paths:
        loader = PyPDFLoader(file_path)
        data = loader.load()

        doc_metadata = [doc.metadata for doc in data]
        doc_content = [doc.page_content for doc in data]

        text_splitter = SentenceTransformersTokenTextSplitter(
            model_name='sentence-transformers/all-mpnet-base-v2',
            chunk_size=100,
            chunk_overlap=50
        )
        chunks = text_splitter.create_documents(doc_content, doc_metadata)
        db.add_documents(chunks)

    return '✅ Documents processed and added to database.'

In [ ]:
# Function to Query using RAG
def run_query(query, groq_api_key):
    retriever = db.as_retriever(search_type='similarity', search_kwargs={'k': 5})
    llm = ChatGroq(model='llama-3.3-70b-versatile', api_key=groq_api_key, temperature=1)

    rag_chain = (
        {'context': retriever | format_docs, 'question': RunnablePassthrough()}
        | prompt_template
        | llm
        | output_parser
    )

    return rag_chain.invoke(query)

In [ ]:
# Example Usage (optional, CLI testing)
# pdf_paths = ['sample1.pdf', 'sample2.pdf']
# process_documents(pdf_paths)

# groq_api_key = 'YOUR_GROQ_API_KEY'
# query = 'What are the AI applications in drug discovery?'
# print(run_query(query, groq_api_key))

In [ ]:
# Gradio Interface
def pharma_query_interface(query, groq_api_key, files):
    if files:
        process_documents(files)
    if not query or not groq_api_key:
        return '⚠️ Please enter a query and your GROQ API key.'
    return run_query(query, groq_api_key)

iface = gr.Interface(
    fn=pharma_query_interface,
    inputs=[
        gr.Textbox(label='Pharmaceutical Question', placeholder='e.g., What are the AI applications in drug discovery?'),
        gr.Textbox(label='Groq API Key', type='password'),
        gr.File(label='Upload PDF documents (optional)', file_types=['.pdf'], file_count='multiple')
    ],
    outputs=gr.Textbox(label='RAG Answer', lines=10),
    title='PharmaAssist - RAG for Pharmaceutical Research',
    description='Upload pharmaceutical research PDFs and ask questions using Groq\'s LLaMA3 and HuggingFace Embeddings.'
)

if __name__ == '__main__':
    iface.launch()